In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("stats (26) - Copy.csv")

df.head()



,Name,player_id,year,age,ab,pa,H,1B,2B,3B,HR,K,BB,RBI,CS,SB,GIDP
0,"Pujols, Albert",405395,2020,40,152,163,34,20,8,0,6,25,9,25,0,0,4
1,"Cabrera, Miguel",408234,2020,37,204,231,51,37,4,0,10,51,24,35,0,1,3
2,"Choo, Shin-Soo",425783,2020,37,110,127,26,18,3,0,5,33,13,15,2,6,1
3,"Molina, Yadier",425877,2020,37,145,156,38,32,2,0,4,21,6,16,0,0,7
4,"Canó, Robinson",429664,2020,37,171,182,54,35,9,0,10,24,9,30,0,0,7


In [2]:
df.info()
df[['age', 'year']].describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2619 entries, 0 to 2618
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Name       2619 non-null   object
 1   player_id  2619 non-null   int64 
 2   year       2619 non-null   int64 
 3   age        2619 non-null   int64 
 4   ab         2619 non-null   int64 
 5   pa         2619 non-null   int64 
 6   H          2619 non-null   int64 
 7   1B         2619 non-null   int64 
 8   2B         2619 non-null   int64 
 9   3B         2619 non-null   int64 
 10  HR         2619 non-null   int64 
 11  K          2619 non-null   int64 
 12  BB         2619 non-null   int64 
 13  RBI        2619 non-null   int64 
 14  CS         2619 non-null   int64 
 15  SB         2619 non-null   int64 
 16  GIDP       2619 non-null   int64 
dtypes: int64(16), object(1)
memory usage: 348.0+ KB


,age,year
count,2619.000000,2619.000000
mean,28.029019,2022.638030
std,3.704714,1.642965
min,20.000000,2020.000000
25%,25.000000,2021.000000
50%,28.000000,2023.000000
75%,30.000000,2024.000000
max,42.000000,2025.000000


In [3]:
counting_stats = [
    'pa',
    'ab',
    'H',
    'HR',
    'RBI',
    'BB',
    'K'
]

age_avg = (
    df
    .groupby('age')[counting_stats]
    .mean()
    .reset_index()
    .sort_values('age')
)

age_avg.head(10)


,age,pa,ab,H,HR,RBI,BB,K
0,20,253.833333,234.500000,59.666667,7.500000,31.666667,16.666667,54.000000
1,21,304.800000,276.628571,70.514286,10.257143,35.485714,23.428571,69.800000
2,22,340.853333,306.066667,76.466667,11.026667,38.666667,28.040000,80.200000
3,23,323.510345,290.800000,71.393103,9.793103,35.296552,26.413793,75.558621
4,24,342.780612,308.918367,76.760204,10.505102,38.755102,26.892857,80.352041
5,25,336.124031,301.806202,74.662791,10.275194,38.244186,27.333333,76.624031
6,26,341.669014,306.320423,76.559859,10.320423,38.633803,28.295775,75.359155
7,27,363.011029,324.750000,81.253676,11.338235,41.272059,30.702206,79.242647
8,28,353.496183,314.522901,77.061069,11.286260,41.576336,31.309160,78.576336
9,29,362.683258,323.642534,80.484163,12.167421,42.832579,31.751131,80.665158


In [4]:
age_deltas = age_avg.copy()

for stat in counting_stats:
    age_deltas[f'{stat}_delta'] = age_deltas[stat].diff()

age_deltas = age_deltas.dropna()
age_deltas = age_deltas[['age'] + [f'{s}_delta' for s in counting_stats]]


In [5]:
age_avg_smooth = age_avg.copy()

for stat in counting_stats:
    age_avg_smooth[f'{stat}_smooth'] = (
        age_avg_smooth[stat]
        .rolling(window=3, center=True, min_periods=1)
        .mean()
    )


In [6]:
df = df.merge(age_deltas, on='age', how='left')


In [7]:
df = df.sort_values(['player_id', 'year'])

df['HR_next'] = df.groupby('player_id')['HR'].shift(-1)

reg_df = df.dropna(subset=['HR_next', 'HR', 'age'])
from sklearn.linear_model import LinearRegression

X = reg_df[['HR', 'age']]
y = reg_df['HR_next']

model = LinearRegression()
model.fit(X, y)

model.coef_, model.intercept_


(array([ 0.59847914, -0.32013874]), np.float64(14.818986355176438))

Projection models

Hitters First

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,r2_score
import ipywidgets as widgets
from IPython.display import display,clear_output

WINDOW_N_FOR_LEADERBOARD=50
MIN_PA_FILTER_DEFAULT=0

def to_scalar(x):
    if isinstance(x,pd.Series): x=x.iloc[0]
    x=pd.to_numeric(x,errors='coerce')
    return float(x) if pd.notna(x) else np.nan

def safe_div(num,den):
    num=pd.to_numeric(num,errors='coerce')
    den=pd.to_numeric(den,errors='coerce')
    return np.where((den>0)&np.isfinite(den)&np.isfinite(num),num/den,np.nan)

def build_X_pred(model,stat_name,current,age_now,pa_val,age_delta):
    cols=list(model.feature_names_in_)
    row={}
    for c in cols:
        if c==stat_name: row[c]=current
        elif c=='age': row[c]=age_now
        elif c=='pa': row[c]=pa_val
        elif c==f'{stat_name}_age_delta': row[c]=age_delta
        else: row[c]=np.nan
    return pd.DataFrame([row],columns=cols)

def add_batting_rates(df_in):
    d=df_in.copy()
    if '1B' not in d.columns and all(c in d.columns for c in ['H','2B','3B','HR']):
        d['1B']=d['H']-d['2B']-d['3B']-d['HR']
    if all(c in d.columns for c in ['1B','2B','3B','HR']):
        d['TB']=d['1B']+2*d['2B']+3*d['3B']+4*d['HR']
    else:
        d['TB']=np.nan
    d['BA']=np.where(d['ab']>0,d['H']/d['ab'],np.nan)
    HBP=d['HBP'] if 'HBP' in d.columns else 0
    SF=d['SF'] if 'SF' in d.columns else 0
    denom=d['ab']+d['BB']+HBP+SF
    d['OBP']=np.where(denom>0,(d['H']+d['BB']+HBP)/denom,np.nan)
    d['SLG']=np.where(d['ab']>0,d['TB']/d['ab'],np.nan)
    d['OPS']=d['OBP']+d['SLG']
    for col in ['BA','OBP','SLG','OPS']:
        if col in d.columns:
            d[col]=pd.to_numeric(d[col],errors='coerce').round(4)
    if 'TB' in d.columns:
        d['TB']=pd.to_numeric(d['TB'],errors='coerce').round(0)
    return d

df=pd.read_csv("stats (26) - Copy.csv").copy()
df=df.loc[:,~df.columns.duplicated()].copy()

required_cols={'Name','player_id','year','age','pa','ab'}
missing=required_cols-set(df.columns)
if missing: raise ValueError(f"Missing required columns: {missing}")

for c in['year','age','pa','ab','player_id']:
    df[c]=pd.to_numeric(df[c],errors='coerce')

df=df.dropna(subset=['player_id','year','age']).copy()
df['year']=df['year'].astype(int)
df['age']=df['age'].astype(int)
df['player_id']=df['player_id'].astype(int)

id_cols=['Name','player_id','year','age']
counting_stats=[c for c in df.columns if c not in id_cols]
for c in counting_stats:
    df[c]=pd.to_numeric(df[c],errors='coerce')

df=add_batting_rates(df)
df=df.sort_values(['player_id','year']).reset_index(drop=True)

rate_stats=[]
if'BB'in df.columns:
    df['BB_per_AB']=safe_div(df['BB'],df['ab'])
    rate_stats.append('BB_per_AB')
if'HR'in df.columns:
    df['HR_per_AB']=safe_div(df['HR'],df['ab'])
    rate_stats.append('HR_per_AB')
if'H'in df.columns:
    df['H_per_AB']=safe_div(df['H'],df['ab'])
    rate_stats.append('H_per_AB')
if'K'in df.columns:
    df['K_per_AB']=safe_div(df['K'],df['ab'])
    rate_stats.append('K_per_AB')

model_stats=[c for c in counting_stats+rate_stats if c in df.columns]

age_avg=df.groupby('age')[model_stats].mean().reset_index().sort_values('age')
age_deltas=age_avg.copy()
for s in model_stats:
    age_deltas[f'{s}_age_delta']=age_deltas[s].diff()
age_deltas=age_deltas[['age']+[f'{s}_age_delta'for s in model_stats]].dropna()
next_age_deltas=age_deltas.rename(columns={'age':'next_age'})

for s in model_stats:
    df[f'{s}_next']=df.groupby('player_id')[s].shift(-1)

train_df=df.dropna(subset=[f'{s}_next'for s in model_stats]).copy()

models={}
metrics_rows=[]
for s in model_stats:
    tmp=train_df[['player_id','year','age','pa',s,f'{s}_next']].copy()
    tmp['next_age']=tmp['age']+1
    tmp=tmp.merge(next_age_deltas[['next_age',f'{s}_age_delta']],on='next_age',how='left').dropna(subset=[f'{s}_age_delta'])
    if len(tmp)<50: 
        continue
    X=tmp[[s,'age','pa',f'{s}_age_delta']]
    y=tmp[f'{s}_next']
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
    m=LinearRegression()
    m.fit(X_train,y_train)
    models[s]=m
    y_pred=m.predict(X_test)
    metrics_rows.append({'stat':s,'MAE':mean_absolute_error(y_test,y_pred),'R2':r2_score(y_test,y_pred)})

model_metrics=pd.DataFrame(metrics_rows).sort_values('R2',ascending=False)

def project_next_year(pid:int):
    hist=df[df['player_id']==pid].sort_values('year').copy()
    if hist.empty: 
        return{},hist,pd.DataFrame(),pd.DataFrame()
    last=hist.iloc[-1]
    age_now=int(last['age']);age_next=age_now+1
    pa_val=to_scalar(last['pa'])
    proj={}
    rows=[]
    for s in model_stats:
        current=to_scalar(last.get(s,np.nan))
        delta_series=age_deltas.loc[age_deltas['age']==age_next,f'{s}_age_delta']
        age_delta=to_scalar(delta_series) if len(delta_series) else np.nan
        age_only=current+age_delta if np.isfinite(current) and np.isfinite(age_delta) else np.nan
        reg=np.nan
        if(s in models) and np.isfinite(current) and np.isfinite(pa_val) and np.isfinite(age_delta):
            X_pred=build_X_pred(models[s],s,current,age_now,pa_val,age_delta)
            reg=float(models[s].predict(X_pred)[0])
        blend=0.5*age_only+0.5*reg if np.isfinite(age_only) and np.isfinite(reg) else np.nan
        proj[s]=blend
        rows.append({'stat':s,'blend':blend,'age_only_proj':age_only,'regression_proj':reg})
    meta={'Name':last['Name'],'player_id':int(last['player_id']),'last_year':int(last['year']),'age_now':age_now,'age_next':age_next,'pa_last':pa_val}
    proj_table=pd.DataFrame(rows).sort_values('stat')

    proj_wide={'Name':meta['Name'],'player_id':meta['player_id'],'last_year':meta['last_year'],'age_next':meta['age_next']}
    for k,v in proj.items():
        proj_wide[f'proj_{k}']=v

    proj_rates=add_batting_rates(pd.DataFrame([{
        'ab':proj.get('ab',np.nan),
        'H':proj.get('H',np.nan),
        'BB':proj.get('BB',np.nan),
        '2B':proj.get('2B',np.nan),
        '3B':proj.get('3B',np.nan),
        'HR':proj.get('HR',np.nan),
        '1B':proj.get('1B',np.nan),
        'HBP':proj.get('HBP',0),
        'SF':proj.get('SF',0)
    }]))

    proj_wide['proj_BA']=round(to_scalar(proj_rates['BA']),4)
    proj_wide['proj_OBP']=round(to_scalar(proj_rates['OBP']),4)
    proj_wide['proj_SLG']=round(to_scalar(proj_rates['SLG']),4)
    proj_wide['proj_OPS']=round(to_scalar(proj_rates['OPS']),4)
    proj_wide['proj_TB']=round(to_scalar(proj_rates['TB']),0)

    for col in list(proj_wide.keys()):
        if col.startswith('proj_') and col not in ['proj_TB']:
            proj_wide[col]=to_scalar(proj_wide[col])
            if np.isfinite(proj_wide[col]): proj_wide[col]=round(proj_wide[col],4)

    proj_wide_df=pd.DataFrame([proj_wide])
    return meta,hist,proj_table,proj_wide_df

def build_all_projections(min_pa_last=0):
    last_rows=df.sort_values(['player_id','year']).groupby('player_id').tail(1)
    last_rows=last_rows[last_rows['pa']>=min_pa_last]
    out=[]
    for _,r in last_rows.iterrows():
        meta,_,_,proj_wide_df=project_next_year(int(r['player_id']))
        if proj_wide_df.empty: 
            continue
        out.append(proj_wide_df.iloc[0].to_dict())
    return pd.DataFrame(out)

ALL_PROJ=build_all_projections(MIN_PA_FILTER_DEFAULT)
proj_cols=sorted([c for c in ALL_PROJ.columns if c.startswith('proj_')])

mode=widgets.ToggleButtons(options=['Player Lookup','Leaderboard'],value='Player Lookup',description='Mode:')
name_box=widgets.Text(value="",placeholder="Type full/partial name",description="Search:",layout=widgets.Layout(width='600px'))
player_dd=widgets.Dropdown(options=[],description="Matches:",layout=widgets.Layout(width='600px'))

stat_dd=widgets.Dropdown(options=proj_cols,description="Sort by:",layout=widgets.Layout(width='600px'))
order_dd=widgets.Dropdown(options=[('Highest','desc'),('Lowest','asc')],value='desc',description="Order:")
n_slider=widgets.IntSlider(value=WINDOW_N_FOR_LEADERBOARD,min=5,max=200,step=5,description="Show N:")
out=widgets.Output()

def get_matches(q,limit=25):
    q=(q or"").strip().lower()
    if not q: return[]
    matches=df[['Name','player_id']].drop_duplicates()
    matches=matches[matches['Name'].str.lower().str.contains(q,na=False)].head(limit)
    return[(f"{row['Name']} (id={int(row['player_id'])})",int(row['player_id']))for _,row in matches.iterrows()]

def refresh_player_matches(change=None):
    opts=get_matches(name_box.value)
    player_dd.options=opts
    if opts: player_dd.value=opts[0][1]

def clean_hist_for_display(hist):
    base=['Name','player_id','year','age','pa','ab']
    stats=['H','1B','2B','3B','HR','BB','K','RBI','SB','CS','GIDP']
    rates=['BA','OBP','SLG','OPS','TB','BB_per_AB','HR_per_AB','H_per_AB','K_per_AB']
    keep=[c for c in base+stats+rates if c in hist.columns]
    out_hist=hist[keep].sort_values('year').copy()
    for c in ['BA','OBP','SLG','OPS','BB_per_AB','HR_per_AB','H_per_AB','K_per_AB']:
        if c in out_hist.columns:
            out_hist[c]=pd.to_numeric(out_hist[c],errors='coerce').round(4)
    if 'TB' in out_hist.columns:
        out_hist['TB']=pd.to_numeric(out_hist['TB'],errors='coerce').round(0)
    return out_hist

def render():
    with out:
        clear_output(wait=True)
        if mode.value=='Player Lookup':
            if player_dd.value is None:
                print("Type a name to search.")
                return
            meta,hist,proj_table,proj_wide_df=project_next_year(int(player_dd.value))
            print(meta)
            print("\n---Last Year Snapshot---")
            display(clean_hist_for_display(hist).tail(1))
            print("\n---Full Prior Years---")
            display(clean_hist_for_display(hist))
            print("\n---Next Year Projections (blend)---")
            display(proj_table[['stat','blend','age_only_proj','regression_proj']])
            print("\n---Projected BA/OBP/SLG/OPS (manual)---")
            show_cols=[c for c in ['proj_BA','proj_OBP','proj_SLG','proj_OPS'] if c in proj_wide_df.columns]
            display(proj_wide_df[['Name','player_id','age_next']+show_cols])
            print("\n---Model quality---")
            display(model_metrics)
        else:
            col=stat_dd.value
            asc=(order_dd.value=='asc')
            n=int(n_slider.value)
            board=ALL_PROJ[['Name','player_id','last_year','age_next',col]].dropna(subset=[col]).sort_values(col,ascending=asc).head(n)
            print(f"Leaderboard sorted by: {col} ({'Lowest' if asc else 'Highest'})")
            display(board)

name_box.observe(refresh_player_matches,names='value')
player_dd.observe(lambda change:render(),names='value')
mode.observe(lambda change:render(),names='value')
stat_dd.observe(lambda change:render(),names='value')
order_dd.observe(lambda change:render(),names='value')
n_slider.observe(lambda change:render(),names='value')

display(widgets.VBox([mode,name_box,player_dd,stat_dd,widgets.HBox([order_dd,n_slider]),out]))
render()


In [21]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,r2_score
import ipywidgets as widgets
from IPython.display import display,clear_output

TARGET_SEASON=2026
WINDOW_N_FOR_LEADERBOARD=50
MIN_PA_FILTER_DEFAULT=0
MIN_ROWS_TO_TRAIN=50

DERIVED_STATS=set(['BA','OBP','SLG','OPS','TB'])

def to_scalar(x):
    if isinstance(x,pd.Series): x=x.iloc[0]
    x=pd.to_numeric(x,errors='coerce')
    return float(x) if pd.notna(x) else np.nan

def safe_div(num,den):
    num=pd.to_numeric(num,errors='coerce')
    den=pd.to_numeric(den,errors='coerce')
    return np.where((den>0)&np.isfinite(den)&np.isfinite(num),num/den,np.nan)

def build_X_pred(model,stat_name,current,age_now,pa_val,age_delta):
    cols=list(model.feature_names_in_)
    row={}
    for c in cols:
        if c==stat_name: row[c]=current
        elif c=='age': row[c]=age_now
        elif c=='pa': row[c]=pa_val
        elif c==f'{stat_name}_age_delta': row[c]=age_delta
        else: row[c]=np.nan
    return pd.DataFrame([row],columns=cols)

def add_batting_rates(df_in):
    d=df_in.copy()
    if '1B' not in d.columns and all(c in d.columns for c in ['H','2B','3B','HR']):
        d['1B']=d['H']-d['2B']-d['3B']-d['HR']
    if all(c in d.columns for c in ['1B','2B','3B','HR']):
        d['TB']=d['1B']+2*d['2B']+3*d['3B']+4*d['HR']
    else:
        d['TB']=np.nan
    d['BA']=np.where(d['ab']>0,d['H']/d['ab'],np.nan)
    HBP=d['HBP'] if 'HBP' in d.columns else 0
    SF=d['SF'] if 'SF' in d.columns else 0
    denom=d['ab']+d['BB']+HBP+SF
    d['OBP']=np.where(denom>0,(d['H']+d['BB']+HBP)/denom,np.nan)
    d['SLG']=np.where(d['ab']>0,d['TB']/d['ab'],np.nan)
    d['OPS']=d['OBP']+d['SLG']
    for col in ['BA','OBP','SLG','OPS']:
        if col in d.columns:
            d[col]=pd.to_numeric(d[col],errors='coerce').round(4)
    if 'TB' in d.columns:
        d['TB']=pd.to_numeric(d['TB'],errors='coerce').round(0)
    return d

def compute_batting_rates_from_counts(p):
    ab=p.get('ab',np.nan)
    H=p.get('H',np.nan)
    BB=p.get('BB',np.nan)
    _2B=p.get('2B',np.nan)
    _3B=p.get('3B',np.nan)
    HR=p.get('HR',np.nan)
    _1B=p.get('1B',np.nan)
    if not np.isfinite(_1B) and np.isfinite(H) and np.isfinite(_2B) and np.isfinite(_3B) and np.isfinite(HR):
        _1B=H-_2B-_3B-HR
    TB=_1B+2*_2B+3*_3B+4*HR if all(np.isfinite(x) for x in [_1B,_2B,_3B,HR]) else np.nan
    BA=H/ab if np.isfinite(H) and np.isfinite(ab) and ab>0 else np.nan
    OBP=(H+BB)/(ab+BB) if np.isfinite(H) and np.isfinite(BB) and np.isfinite(ab) and (ab+BB)>0 else np.nan
    SLG=TB/ab if np.isfinite(TB) and np.isfinite(ab) and ab>0 else np.nan
    OPS=OBP+SLG if np.isfinite(OBP) and np.isfinite(SLG) else np.nan
    out={'BA':BA,'OBP':OBP,'SLG':SLG,'OPS':OPS,'TB':TB}
    for k in out:
        out[k]=round(out[k],4) if np.isfinite(out[k]) else np.nan
    if np.isfinite(out['TB']): out['TB']=round(out['TB'],0)
    return out

df=pd.read_csv("stats (26) - Copy.csv").copy()
df=df.loc[:,~df.columns.duplicated()].copy()

required_cols={'Name','player_id','year','age','pa','ab'}
missing=required_cols-set(df.columns)
if missing: raise ValueError(f"Missing required columns: {missing}")

for c in['year','age','pa','ab','player_id']:
    df[c]=pd.to_numeric(df[c],errors='coerce')

df=df.dropna(subset=['player_id','year','age']).copy()
df['year']=df['year'].astype(int)
df['age']=df['age'].astype(int)
df['player_id']=df['player_id'].astype(int)

id_cols=['Name','player_id','year','age']
counting_stats=[c for c in df.columns if c not in id_cols]
for c in counting_stats:
    df[c]=pd.to_numeric(df[c],errors='coerce')

df=add_batting_rates(df)
df=df.sort_values(['player_id','year']).reset_index(drop=True)

rate_stats=[]
if'BB'in df.columns:
    df['BB_per_AB']=safe_div(df['BB'],df['ab'])
    rate_stats.append('BB_per_AB')
if'HR'in df.columns:
    df['HR_per_AB']=safe_div(df['HR'],df['ab'])
    rate_stats.append('HR_per_AB')
if'H'in df.columns:
    df['H_per_AB']=safe_div(df['H'],df['ab'])
    rate_stats.append('H_per_AB')
if'K'in df.columns:
    df['K_per_AB']=safe_div(df['K'],df['ab'])
    rate_stats.append('K_per_AB')

model_stats=[c for c in counting_stats+rate_stats if c in df.columns]

age_avg=df.groupby('age')[model_stats].mean().reset_index().sort_values('age')
age_deltas=age_avg.copy()
for s in model_stats:
    age_deltas[f'{s}_age_delta']=age_deltas[s].diff()
age_deltas=age_deltas[['age']+[f'{s}_age_delta'for s in model_stats]].dropna()
next_age_deltas=age_deltas.rename(columns={'age':'next_age'})

for s in model_stats:
    df[f'{s}_next']=df.groupby('player_id')[s].shift(-1)

train_df=df.dropna(subset=[f'{s}_next'for s in model_stats]).copy()

models={}
metrics_rows=[]
for s in model_stats:
    tmp=train_df[['player_id','year','age','pa',s,f'{s}_next']].copy()
    tmp['next_age']=tmp['age']+1
    tmp=tmp.merge(next_age_deltas[['next_age',f'{s}_age_delta']],on='next_age',how='left').dropna(subset=[f'{s}_age_delta'])
    if len(tmp)<MIN_ROWS_TO_TRAIN:
        continue
    X=tmp[[s,'age','pa',f'{s}_age_delta']]
    y=tmp[f'{s}_next']
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
    m=LinearRegression()
    m.fit(X_train,y_train)
    models[s]=m
    y_pred=m.predict(X_test)
    metrics_rows.append({'stat':s,'MAE':mean_absolute_error(y_test,y_pred),'R2':r2_score(y_test,y_pred),'n':len(tmp)})

model_metrics=pd.DataFrame(metrics_rows).sort_values('R2',ascending=False)

def project_next_year(pid:int):
    hist=df[df['player_id']==pid].sort_values('year').copy()
    if hist.empty:
        return{},hist,pd.DataFrame(),pd.DataFrame()
    last=hist.iloc[-1]
    age_now=int(last['age']);age_next=age_now+1
    pa_val=to_scalar(last['pa'])

    proj={}
    rows=[]

    for s in model_stats:
        current=to_scalar(last.get(s,np.nan))
        delta_series=age_deltas.loc[age_deltas['age']==age_next,f'{s}_age_delta']
        age_delta=to_scalar(delta_series) if len(delta_series) else np.nan
        age_only=current+age_delta if np.isfinite(current) and np.isfinite(age_delta) else np.nan
        reg=np.nan
        if(s in models) and np.isfinite(current) and np.isfinite(pa_val) and np.isfinite(age_delta):
            X_pred=build_X_pred(models[s],s,current,age_now,pa_val,age_delta)
            reg=float(models[s].predict(X_pred)[0])
        blend=0.5*age_only+0.5*reg if np.isfinite(age_only) and np.isfinite(reg) else np.nan
        proj[s]=blend
        rows.append({'stat':s,'blend':blend,'age_only_proj':age_only,'regression_proj':reg})

    derived=compute_batting_rates_from_counts(proj)
    for k,v in derived.items():
        proj[k]=v
    for k in ['BA','OBP','SLG','OPS','TB']:
        rows.append({'stat':k,'blend':proj.get(k,np.nan),'age_only_proj':np.nan,'regression_proj':np.nan})

    proj_table=pd.DataFrame(rows)
    for c in ['blend','age_only_proj','regression_proj']:
        proj_table[c]=pd.to_numeric(proj_table[c],errors='coerce').round(4)
    proj_table=proj_table.sort_values('stat').reset_index(drop=True)

    proj_table_display=proj_table.copy()
    proj_table_display.loc[proj_table_display['stat'].isin(DERIVED_STATS),
                       ['age_only_proj','regression_proj']] = np.nan

    meta={'Name':last['Name'],'player_id':int(last['player_id']),'last_year':int(last['year']),'age_now':age_now,'age_next':age_next,'pa_last':pa_val}

    proj_wide={'Name':meta['Name'],'player_id':meta['player_id'],'last_year':meta['last_year'],'age_next':meta['age_next']}
    for k,v in proj.items():
        proj_wide[f'proj_{k}']=v

    proj_wide_df=pd.DataFrame([proj_wide])
    return meta,hist,proj_table_display,proj_wide_df

def build_all_projections(target_season=TARGET_SEASON,min_pa_last=0):
    last_rows=df.sort_values(['player_id','year']).groupby('player_id').tail(1).copy()
    last_rows=last_rows[last_rows['pa']>=min_pa_last]
    last_rows=last_rows[last_rows['year']==target_season-1]
    out=[]
    for _,r in last_rows.iterrows():
        _,_,_,w=project_next_year(int(r['player_id']))
        if w.empty:
            continue
        row=w.iloc[0].to_dict()
        row['proj_year']=target_season
        out.append(row)
    return pd.DataFrame(out)

ALL_PROJ=build_all_projections(TARGET_SEASON,MIN_PA_FILTER_DEFAULT)
proj_cols=sorted([c for c in ALL_PROJ.columns if c.startswith('proj_')])

mode=widgets.ToggleButtons(options=['Player Lookup','Leaderboard'],value='Player Lookup',description='Mode:')
name_box=widgets.Text(value="",placeholder="Type full/partial name",description="Search:",layout=widgets.Layout(width='600px'))
player_dd=widgets.Dropdown(options=[],description="Matches:",layout=widgets.Layout(width='600px'))
stat_dd=widgets.Dropdown(options=proj_cols,description="Sort by:",layout=widgets.Layout(width='600px'))
order_dd=widgets.Dropdown(options=[('Highest','desc'),('Lowest','asc')],value='desc',description="Order:")
n_slider=widgets.IntSlider(value=WINDOW_N_FOR_LEADERBOARD,min=5,max=200,step=5,description="Show N:")
min_pa_box=widgets.IntText(value=MIN_PA_FILTER_DEFAULT,description="Min PA(last):")
rebuild_btn=widgets.Button(description="Rebuild",button_style='info')
out=widgets.Output()

def get_matches(q,limit=25):
    q=(q or"").strip().lower()
    if not q: return[]
    matches=df[['Name','player_id']].drop_duplicates()
    matches=matches[matches['Name'].str.lower().str.contains(q,na=False)].head(limit)
    return[(f"{row['Name']} (id={int(row['player_id'])})",int(row['player_id']))for _,row in matches.iterrows()]

def refresh_player_matches(change=None):
    opts=get_matches(name_box.value)
    player_dd.options=opts
    if opts: player_dd.value=opts[0][1]

def clean_hist_for_display(hist):
    base=['Name','player_id','year','age','pa','ab']
    stats=['H','1B','2B','3B','HR','BB','K','RBI','SB','CS','GIDP']
    rates=['BA','OBP','SLG','OPS','TB','BB_per_AB','HR_per_AB','H_per_AB','K_per_AB']
    keep=[c for c in base+stats+rates if c in hist.columns]
    out_hist=hist[keep].sort_values('year').copy()
    for c in ['BA','OBP','SLG','OPS','BB_per_AB','HR_per_AB','H_per_AB','K_per_AB']:
        if c in out_hist.columns:
            out_hist[c]=pd.to_numeric(out_hist[c],errors='coerce').round(4)
    if 'TB' in out_hist.columns:
        out_hist['TB']=pd.to_numeric(out_hist['TB'],errors='coerce').round(0)
    return out_hist

def rebuild(btn=None):
    global ALL_PROJ,proj_cols
    with out:
        clear_output(wait=True)
        print(f"Rebuilding {TARGET_SEASON} leaderboard...")
    ALL_PROJ=build_all_projections(TARGET_SEASON,int(min_pa_box.value))
    proj_cols=sorted([c for c in ALL_PROJ.columns if c.startswith('proj_')])
    stat_dd.options=proj_cols
    if proj_cols: stat_dd.value=proj_cols[0]
    render()

def render():
    with out:
        clear_output(wait=True)
        if mode.value=='Player Lookup':
            if player_dd.value is None:
                print("Type a name to search.")
                return
            meta,hist,proj_table_display,proj_wide_df=project_next_year(int(player_dd.value))
            print(meta)
            print("\n---Last Year Snapshot---")
            display(clean_hist_for_display(hist).tail(1))
            print("\n---Full Prior Years---")
            display(clean_hist_for_display(hist))
            print("\n---Next Year Projections (blend)---")
            display(
    proj_table_display.style.format(
        {"age_only_proj": "{:.4f}",
         "regression_proj": "{:.4f}",
         "blend": "{:.4f}"},
        na_rep=""
    )
)
            print("\n---Model quality---")
            display(model_metrics)
        else:
            if stat_dd.value is None:
                print("No proj_* columns available.")
                return
            col=stat_dd.value
            asc=(order_dd.value=='asc')
            n=int(n_slider.value)
            board=ALL_PROJ[['Name','player_id','proj_year',col]].dropna(subset=[col]).sort_values(col,ascending=asc).head(n)
            print(f"{TARGET_SEASON} Leaderboard sorted by: {col} ({'Lowest' if asc else 'Highest'})")
            display(board)

name_box.observe(refresh_player_matches,names='value')
player_dd.observe(lambda change:render(),names='value')
mode.observe(lambda change:render(),names='value')
stat_dd.observe(lambda change:render(),names='value')
order_dd.observe(lambda change:render(),names='value')
n_slider.observe(lambda change:render(),names='value')
rebuild_btn.on_click(rebuild)

display(widgets.VBox([mode,name_box,player_dd,stat_dd,widgets.HBox([order_dd,n_slider]),widgets.HBox([min_pa_box,rebuild_btn]),out]))
render()


Pitchers Now

In [15]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,r2_score
import ipywidgets as widgets
from IPython.display import display,clear_output

WINDOW_N_FOR_LEADERBOARD=50
MIN_IP_FILTER_DEFAULT=0

def to_scalar(x):
    if isinstance(x,pd.Series): x=x.iloc[0]
    x=pd.to_numeric(x,errors='coerce')
    return float(x) if pd.notna(x) else np.nan

def safe_div(num,den):
    num=pd.to_numeric(num,errors='coerce')
    den=pd.to_numeric(den,errors='coerce')
    return np.where((den>0)&np.isfinite(den)&np.isfinite(num),num/den,np.nan)

def build_X_pred(model,stat_name,current,age_now,ip_val,age_delta):
    cols=list(model.feature_names_in_)
    row={}
    for c in cols:
        if c==stat_name: row[c]=current
        elif c=='age': row[c]=age_now
        elif c=='IP': row[c]=ip_val
        elif c==f'{stat_name}_age_delta': row[c]=age_delta
        else: row[c]=np.nan
    return pd.DataFrame([row],columns=cols)

def build_pitcher_name(df):
    #Your file uses a single column: "last_name, first_name"
    if 'last_name, first_name' in df.columns:
        df['Name']=df['last_name, first_name'].astype(str)
        return df
    #Fallbacks if you ever switch formats
    if 'last_name' in df.columns and 'first_name' in df.columns:
        df['Name']=df['last_name'].astype(str)+", "+df['first_name'].astype(str)
        return df
    if 'Name' in df.columns:
        return df
    raise ValueError("Missing pitcher name column. Expected 'last_name, first_name'.")

def parse_ip(x):
    #Handles "123.1" style (1=1/3 inning, 2=2/3 inning) or numeric already
    if pd.isna(x): 
        return np.nan
    try:
        val=float(x)
    except:
        return np.nan
    whole=int(val)
    frac=val-whole
    if abs(frac-0.1)<1e-6:
        return whole+1/3
    if abs(frac-0.2)<1e-6:
        return whole+2/3
    return val

def add_pitching_rates(df_in):
    d=df_in.copy()
    #Your columns from screenshot
    if 'p_formatted_ip' in d.columns:
        d['IP']=d['p_formatted_ip'].apply(parse_ip)
    else:
        d['IP']=np.nan

    if 'p_earned_run' in d.columns:
        d['ER']=pd.to_numeric(d['p_earned_run'],errors='coerce')
    else:
        d['ER']=np.nan

    if 'hit' in d.columns:
        d['H']=pd.to_numeric(d['hit'],errors='coerce')
    else:
        d['H']=np.nan

    if 'walk' in d.columns:
        d['BB']=pd.to_numeric(d['walk'],errors='coerce')
    else:
        d['BB']=np.nan

    if 'strikeout' in d.columns:
        d['SO']=pd.to_numeric(d['strikeout'],errors='coerce')
    else:
        d['SO']=np.nan

    if 'home_run' in d.columns:
        d['HR']=pd.to_numeric(d['home_run'],errors='coerce')
    else:
        d['HR']=np.nan

    #Use provided ERA if present, else compute
    if 'p_era' in d.columns:
        d['ERA']=pd.to_numeric(d['p_era'],errors='coerce')
    else:
        d['ERA']=np.where(d['IP']>0,(d['ER']*9)/d['IP'],np.nan)

    d['WHIP']=np.where(d['IP']>0,(d['BB']+d['H'])/d['IP'],np.nan)
    d['K9']=np.where(d['IP']>0,(d['SO']*9)/d['IP'],np.nan)
    d['BB9']=np.where(d['IP']>0,(d['BB']*9)/d['IP'],np.nan)
    d['HR9']=np.where(d['IP']>0,(d['HR']*9)/d['IP'],np.nan)
    d['K_BB']=np.where(d['BB']>0,d['SO']/d['BB'],np.nan)

    for c in ['ERA','WHIP','K9','BB9','HR9','K_BB']:
        if c in d.columns:
            d[c]=pd.to_numeric(d[c],errors='coerce').round(4)
    if 'IP' in d.columns:
        d['IP']=pd.to_numeric(d['IP'],errors='coerce').round(1)
    return d

def compute_pitching_rates_from_proj(p):
    IP=p.get('IP',np.nan)
    ER=p.get('ER',np.nan)
    H=p.get('H',np.nan)
    BB=p.get('BB',np.nan)
    SO=p.get('SO',np.nan)
    HR=p.get('HR',np.nan)
    ERA=(ER*9)/IP if np.isfinite(ER) and np.isfinite(IP) and IP>0 else np.nan
    WHIP=(BB+H)/IP if np.isfinite(BB) and np.isfinite(H) and np.isfinite(IP) and IP>0 else np.nan
    K9=(SO*9)/IP if np.isfinite(SO) and np.isfinite(IP) and IP>0 else np.nan
    BB9=(BB*9)/IP if np.isfinite(BB) and np.isfinite(IP) and IP>0 else np.nan
    HR9=(HR*9)/IP if np.isfinite(HR) and np.isfinite(IP) and IP>0 else np.nan
    K_BB=SO/BB if np.isfinite(SO) and np.isfinite(BB) and BB>0 else np.nan
    out={'ERA':ERA,'WHIP':WHIP,'K9':K9,'BB9':BB9,'HR9':HR9,'K_BB':K_BB}
    for k in out:
        out[k]=round(out[k],4) if np.isfinite(out[k]) else np.nan
    return out

df=pd.read_csv("stats (29).csv").copy()
df=df.loc[:,~df.columns.duplicated()].copy()

df=build_pitcher_name(df)

#Rename columns to standard
rename_map={}
if 'player_age' in df.columns: rename_map['player_age']='age'
if 'year' in df.columns: rename_map['year']='year'
if 'player_id' in df.columns: rename_map['player_id']='player_id'
df=df.rename(columns=rename_map)

required={'Name','player_id','year','age'}
missing=required-set(df.columns)
if missing:
    raise ValueError(f"Missing required columns in pitcher CSV: {missing}")

for c in ['player_id','year','age']:
    df[c]=pd.to_numeric(df[c],errors='coerce')

df=df.dropna(subset=['player_id','year','age']).copy()
df['player_id']=df['player_id'].astype(int)
df['year']=df['year'].astype(int)
df['age']=df['age'].astype(int)

df=add_pitching_rates(df)
df=df.sort_values(['player_id','year']).reset_index(drop=True)

#Choose model stats based on your file
counting_core=[c for c in ['IP','ER','H','BB','SO','HR'] if c in df.columns]
rate_core=[c for c in ['ERA','WHIP','K9','BB9','HR9','K_BB'] if c in df.columns]
model_stats=[c for c in (counting_core+rate_core) if c in df.columns]

if len(model_stats)==0:
    raise ValueError("No numeric pitching stats found to model. Check your column names.")

age_avg=df.groupby('age')[model_stats].mean().reset_index().sort_values('age')
age_deltas=age_avg.copy()
for s in model_stats:
    age_deltas[f'{s}_age_delta']=age_deltas[s].diff()
age_deltas=age_deltas[['age']+[f'{s}_age_delta'for s in model_stats]].dropna()
next_age_deltas=age_deltas.rename(columns={'age':'next_age'})

for s in model_stats:
    df[f'{s}_next']=df.groupby('player_id')[s].shift(-1)

train_df=df.dropna(subset=[f'{s}_next'for s in model_stats]).copy()

models={}
metrics_rows=[]
for s in model_stats:
    tmp=train_df[['player_id','year','age','IP',s,f'{s}_next']].copy()
    tmp=tmp.dropna(subset=['IP'])
    tmp['next_age']=tmp['age']+1
    tmp=tmp.merge(next_age_deltas[['next_age',f'{s}_age_delta']],on='next_age',how='left').dropna(subset=[f'{s}_age_delta'])
    if len(tmp)<50:
        continue
    X=tmp[[s,'age','IP',f'{s}_age_delta']]
    y=tmp[f'{s}_next']
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
    m=LinearRegression()
    m.fit(X_train,y_train)
    models[s]=m
    y_pred=m.predict(X_test)
    metrics_rows.append({'stat':s,'MAE':mean_absolute_error(y_test,y_pred),'R2':r2_score(y_test,y_pred)})

model_metrics=pd.DataFrame(metrics_rows).sort_values('R2',ascending=False)

def clean_hist_for_display(hist):
    base=['Name','player_id','year','age','IP']
    stats=[c for c in ['ER','H','BB','SO','HR'] if c in hist.columns]
    rates=[c for c in ['ERA','WHIP','K9','BB9','HR9','K_BB'] if c in hist.columns]
    extra=[c for c in ['p_win','p_loss','p_quality_start'] if c in hist.columns]
    keep=[c for c in base+stats+rates+extra if c in hist.columns]
    out=hist[keep].sort_values('year').copy()
    for c in ['ERA','WHIP','K9','BB9','HR9','K_BB']:
        if c in out.columns:
            out[c]=pd.to_numeric(out[c],errors='coerce').round(4)
    if 'IP' in out.columns:
        out['IP']=pd.to_numeric(out['IP'],errors='coerce').round(1)
    return out

def project_next_year(pid:int):
    hist=df[df['player_id']==pid].sort_values('year').copy()
    if hist.empty:
        return{},hist,pd.DataFrame(),pd.DataFrame()
    last=hist.iloc[-1]
    age_now=int(last['age']);age_next=age_now+1
    ip_val=to_scalar(last.get('IP',np.nan))
    proj={}
    rows=[]
    for s in model_stats:
        current=to_scalar(last.get(s,np.nan))
        delta_series=age_deltas.loc[age_deltas['age']==age_next,f'{s}_age_delta']
        age_delta=to_scalar(delta_series) if len(delta_series) else np.nan
        age_only=current+age_delta if np.isfinite(current) and np.isfinite(age_delta) else np.nan
        reg=np.nan
        if (s in models) and np.isfinite(current) and np.isfinite(ip_val) and np.isfinite(age_delta):
            X_pred=build_X_pred(models[s],s,current,age_now,ip_val,age_delta)
            reg=float(models[s].predict(X_pred)[0])
        blend=0.5*age_only+0.5*reg if np.isfinite(age_only) and np.isfinite(reg) else np.nan
        proj[s]=blend
        rows.append({'stat':s,'blend':blend,'age_only_proj':age_only,'regression_proj':reg})
    meta={'Name':last['Name'],'player_id':int(last['player_id']),'last_year':int(last['year']),'age_now':age_now,'age_next':age_next,'IP_last':ip_val}
    proj_table=pd.DataFrame(rows).sort_values('stat')

    derived=compute_pitching_rates_from_proj(proj)
    for k,v in derived.items():
        proj[k]=v
    extra_rows=[]
    for k in ['ERA','WHIP','K9','BB9','HR9','K_BB']:
        if k in proj:
            extra_rows.append({'stat':k,'blend':proj[k],'age_only_proj':np.nan,'regression_proj':np.nan})
    if len(extra_rows):
        proj_table=pd.concat([proj_table,pd.DataFrame(extra_rows)],ignore_index=True).sort_values('stat')

    proj_wide={'Name':meta['Name'],'player_id':meta['player_id'],'last_year':meta['last_year'],'age_next':meta['age_next']}
    for k,v in proj.items():
        proj_wide[f'proj_{k}']=v
    for c in list(proj_wide.keys()):
        if c.startswith('proj_'):
            val=proj_wide[c]
            if isinstance(val,(int,float,np.floating)) and np.isfinite(val):
                proj_wide[c]=round(float(val),4)
    if 'proj_IP' in proj_wide and np.isfinite(proj_wide['proj_IP']):
        proj_wide['proj_IP']=round(float(proj_wide['proj_IP']),1)

    proj_wide_df=pd.DataFrame([proj_wide])
    return meta,hist,proj_table,proj_wide_df

def build_all_projections(min_ip_last=0):
    last_rows=df.sort_values(['player_id','year']).groupby('player_id').tail(1)
    if 'IP' in last_rows.columns:
        last_rows=last_rows[last_rows['IP']>=min_ip_last]
    out=[]
    for _,r in last_rows.iterrows():
        meta,_,_,proj_wide_df=project_next_year(int(r['player_id']))
        if proj_wide_df.empty:
            continue
        out.append(proj_wide_df.iloc[0].to_dict())
    return pd.DataFrame(out)

ALL_PROJ=build_all_projections(MIN_IP_FILTER_DEFAULT)
proj_cols=sorted([c for c in ALL_PROJ.columns if c.startswith('proj_')])

mode=widgets.ToggleButtons(options=['Player Lookup','Leaderboard'],value='Player Lookup',description='Mode:')
name_box=widgets.Text(value="",placeholder="Type full/partial name",description="Search:",layout=widgets.Layout(width='600px'))
player_dd=widgets.Dropdown(options=[],description="Matches:",layout=widgets.Layout(width='600px'))
stat_dd=widgets.Dropdown(options=proj_cols,description="Sort by:",layout=widgets.Layout(width='600px'))
order_dd=widgets.Dropdown(options=[('Highest','desc'),('Lowest','asc')],value='desc',description="Order:")
n_slider=widgets.IntSlider(value=WINDOW_N_FOR_LEADERBOARD,min=5,max=200,step=5,description="Show N:")
min_ip_box=widgets.IntText(value=MIN_IP_FILTER_DEFAULT,description="Min IP(last):")
rebuild_btn=widgets.Button(description="Rebuild",button_style='info')
out=widgets.Output()

def get_matches(q,limit=25):
    q=(q or"").strip().lower()
    if not q: return[]
    matches=df[['Name','player_id']].drop_duplicates()
    matches=matches[matches['Name'].str.lower().str.contains(q,na=False)].head(limit)
    return[(f"{row['Name']} (id={int(row['player_id'])})",int(row['player_id']))for _,row in matches.iterrows()]

def refresh_player_matches(change=None):
    opts=get_matches(name_box.value)
    player_dd.options=opts
    if opts: player_dd.value=opts[0][1]

def rebuild(btn=None):
    global ALL_PROJ,proj_cols
    with out:
        clear_output(wait=True)
        print("Rebuilding leaderboard...")
    ALL_PROJ=build_all_projections(int(min_ip_box.value))
    proj_cols=sorted([c for c in ALL_PROJ.columns if c.startswith('proj_')])
    stat_dd.options=proj_cols
    if proj_cols: stat_dd.value=proj_cols[0]
    render()

def render():
    with out:
        clear_output(wait=True)
        if mode.value=='Player Lookup':
            if player_dd.value is None:
                print("Type a name to search.")
                return
            meta,hist,proj_table,proj_wide_df=project_next_year(int(player_dd.value))
            print(meta)
            print("\n---Last Year Snapshot---")
            display(clean_hist_for_display(hist).tail(1))
            print("\n---Full Prior Years---")
            display(clean_hist_for_display(hist))
            print("\n---Next Year Projections (blend)---")
            show=proj_table[['stat','blend','age_only_proj','regression_proj']].copy()
            for c in ['blend','age_only_proj','regression_proj']:
                show[c]=pd.to_numeric(show[c],errors='coerce').round(4)
            display(show.sort_values('stat'))
            print("\n---Wide Projection Row---")
            display(proj_wide_df)
            print("\n---Model quality---")
            display(model_metrics)
        else:
            if stat_dd.value is None:
                print("No proj_* columns available.")
                return
            col=stat_dd.value
            asc=(order_dd.value=='asc')
            n=int(n_slider.value)
            board=ALL_PROJ[['Name','player_id','last_year','age_next',col]].dropna(subset=[col]).sort_values(col,ascending=asc).head(n)
            print(f"Leaderboard sorted by: {col} ({'Lowest' if asc else 'Highest'})")
            display(board)

name_box.observe(refresh_player_matches,names='value')
player_dd.observe(lambda change:render(),names='value')
mode.observe(lambda change:render(),names='value')
stat_dd.observe(lambda change:render(),names='value')
order_dd.observe(lambda change:render(),names='value')
n_slider.observe(lambda change:render(),names='value')
rebuild_btn.on_click(rebuild)

display(widgets.VBox([mode,name_box,player_dd,stat_dd,widgets.HBox([order_dd,n_slider]),widgets.HBox([min_ip_box,rebuild_btn]),out]))
render()
